In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
#loading data
test_data = pd.read_csv("C:\Users\horch\Desktop\Data mining\Titanic\_test.csv")
train_data = pd.read_csv("C:\Users\horch\Desktop\Data mining\Titanic\_train.csv")
data = pd.concat([train_data, test_data])
test_data.shape

(418, 11)

In [5]:
#добавляем колонку с титулами
S = []
T = []
title = []
for i in xrange(len(data['Name'])):
    S.append(str(np.array(data['Name'])[i]).split(','))
    T.append(str(S[i][1]).split('.'))
    title.append(T[i][0])
title = pd.DataFrame(title, columns = ['Title'])
data_new = data.join(title).sort_values(by = 'PassengerId')

In [6]:
#Заполняем NA в возрасте и порте отправки
def fillna_age(train):
    for i in [train]:
        i['Age'] = train.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
    return train
data_new = fillna_age(data_new)
#заполнили NA в порту отправки большинством
data_new['Embarked'] = data_new['Embarked'].fillna('S')
data_new['Fare'] = data_new['Fare'].fillna(0.)
# заполнили нулевой тариф медианами по классам 
for i in xrange(len(data_new['Fare'].values)):
    if data_new['Fare'].values[i] == 0.:
        if data_new['Pclass'].values[i] == 1:
            data_new['Fare'].values[i] = data_new.groupby('Pclass')['Fare'].median()[1]
        if data_new['Pclass'].values[i] == 2:
            data_new['Fare'].values[i] = data_new.groupby('Pclass')['Fare'].median()[2]
        if data_new['Pclass'].values[i] == 3:
            data_new['Fare'].values[i] = data_new.groupby('Pclass')['Fare'].median()[3]
            
data_new[1043:1047]            

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
152,60.5,NaN,S,8.0500,"Storey, Mr. Thomas",0,1044,3,male,0,NaN,3701,Mr
153,36.0,NaN,S,12.1833,"Klasen, Mrs. (Hulda Kristina Eugenia Lofqvist)",2,1045,3,female,0,NaN,350405,Mr
154,13.0,NaN,S,31.3875,"Asplund, Master. Filip Oscar",2,1046,3,male,4,NaN,347077,Mr
155,24.0,NaN,S,7.5500,"Duquemin, Mr. Joseph",0,1047,3,male,0,NaN,S.O./P.P. 752,Mr


In [44]:
#вводим колонку "семья", агрегирующую родственников 
data_new['Family'] = data_new['Parch'] + data_new['SibSp']

#вводим колонку наличие родственников в принципе
data_new['isFamily'] = data_new['Family']
for i in xrange(len(data_new['Family'])):
    if data_new['Family'].values[i] > 0:
        data_new['isFamily'].values[i] = 1
    else: data_new['isFamily'].values[i] = 0
        
# кодируем пол
data_new['Male'] = data_new['Sex']
data_new['Female'] = data_new['Sex']
for i in xrange(len(data_new['Sex'])):
    if data_new['Sex'].values[i] == 'male':
        data_new['Male'].values[i] = 1
    else: data_new['Male'].values[i] = 0
    if data_new['Male'].values[i] == 0:
        data_new['Female'].values[i] = 1
    else: data_new['Female'].values[i] = 0            

# кодируем наличие кабины
data_new['isCabin'] = data_new['Cabin'].fillna(0)
for i in xrange(len(data_new['Cabin'])):
    if data_new['Cabin'].values[i] >0:
        data_new['isCabin'].values[i] = 1
    else: data_new['isCabin'].values[i] = 0        

In [6]:
from sklearn.feature_extraction import DictVectorizer as DV

In [45]:
num_cols = ['Pclass', 'Age', 'Fare', 'Family', 'isFamily'] # вещественные признаки для отбора в модель
cat_cols = ['Male', 'Female', 'isCabin', 'enEmbarked_C', 'enEmbarked_Q', 'enEmbarked_S', 'Capt', 'Col', 'Don', 'Dr', 
            'Jonkheer', 'Lady', 'Major', 'Master', 'Miss', 'Mlle', 
              'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir', 'the Countess', 'unknown' 
           ]# категориальные признаки для отбора в модель

In [46]:
embarked = data_new['Embarked'].values
for i in xrange(len(embarked)):
    s = embarked[i]
    embarked[i] = str(s)       
encoder = DV(sparse = False)
embarked_encoded = encoder.fit_transform(pd.DataFrame(embarked).T.to_dict().values())# закодировали ембаркед
title_encoded = encoder.fit_transform(title.T.to_dict().values())# закодировали титул
title_encoded[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.])

In [47]:
enEmbarked = pd.DataFrame(embarked_encoded, columns = [['enEmbarked_C','enEmbarked_Q', 'enEmbarked_S']])
enTitle = pd.DataFrame(title_encoded, columns = [['Capt', 'Col', 'Don', 'Dr', 
            'Jonkheer', 'Lady', 'Major', 'Master', 'Miss', 'Mlle', 
              'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir', 'the Countess', 'unknown']] )
data_new = data_new.join(enEmbarked).join(enTitle).sort_values(by = 'PassengerId')
data_to_class = data_new[cat_cols+num_cols] #итоговая таблица!!!

In [48]:
#делим на обучение и тест
train_data_to_class = data_to_class[:891]
test_data_to_class = data_to_class[891:]
train_labels = data['Survived'][:891]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 29 columns):
Male            418 non-null object
Female          418 non-null object
isCabin         418 non-null object
enEmbarked_C    418 non-null float64
enEmbarked_Q    418 non-null float64
enEmbarked_S    418 non-null float64
Capt            418 non-null float64
Col             418 non-null float64
Don             418 non-null float64
Dr              418 non-null float64
Jonkheer        418 non-null float64
Lady            418 non-null float64
Major           418 non-null float64
Master          418 non-null float64
Miss            418 non-null float64
Mlle            418 non-null float64
Mme             418 non-null float64
Mr              418 non-null float64
Mrs             418 non-null float64
Ms              418 non-null float64
Rev             418 non-null float64
Sir             418 non-null float64
the Countess    418 non-null float64
unknown         418 non-null float64
Pclass    

In [57]:
#создаем рандомный лес
from sklearn import ensemble, cross_validation, metrics, grid_search


cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 10, test_size = 0.2, random_state = 0)
classifier = ensemble.RandomForestClassifier(random_state = 0, n_estimators = 500, max_depth = 10, min_samples_split=8, min_samples_leaf=2)
cross_validation.cross_val_score(classifier, train_data_to_class, train_labels, cv=cv).mean()

0.8329608938547487

In [58]:
classifier.fit(train_data_to_class, train_labels)
predictions = classifier.predict(test_data_to_class)

In [3]:
submission = pd.DataFrame({"PassengerId": test_data["PassengerId"],"Survived": predictions})
submission.to_csv("titanic-submission.csv", index=False)
#pd.DataFrame(classifier.predict(test_data_to_class)).to_csv('result.csv', sep=',', header=False, index=False)

NameError: name 'predictions' is not defined